# Quantum Disambiguatation

In [ ]:
from discopy import Ty, Word
from words import sub_nouns as nouns
from words import sub_sentences as sentences

s, n = Ty('s'), Ty('n')

dataset = []
nouns, verbs = set(), set()
for sentence in sentences:
    _s = sentence.split(" ")
    dataset.append([_s[0],_s[1],int(_s[2][1])])
    verbs.add(_s[0])
    nouns.add(_s[1])


ambiguous_verbs = ["file"] #, "dribble", "tap", "charge"]


# nouns
noun_voc = {noun: Word(noun, n) for noun in nouns}
unamb_verb_voc = {verb: Word(verb, s @ n.l) for verb in verbs if verb not in ambiguous_verbs}
amb_verb_voc = {amb: Word(amb, s @ n.l) for amb in ambiguous_verbs}

In [ ]:
print(unamb_verb_voc)
print(amb_verb_voc)

## Prepare and parse dataset

In [ ]:
from time import time
from discopy import Diagram, Id, Cup
from discopy.grammar import brute_force

grammar = Id(s) @ Cup(n.l, n)
sentences, plausability, parsing = list(), list(), dict()

start = time()
for entry in dataset:
    sentence = ' '.join(entry[0:2]) + '.'
    sentences.append(sentence)
    plausability.append(entry[2])

    verb = unamb_verb_voc.get(entry[0], False)
    if not verb:
        verb = amb_verb_voc[entry[0]]
    obj = noun_voc[entry[1]]
    diagram = verb @ obj >> grammar

    parsing.update({sentence: diagram})


In [ ]:
# display a random sentence
i = 10
print(sentences[i]," plausability: ", plausability[i])
parsing[sentences[i]].draw(draw_type_labels=True)

In [ ]:
from discopy.quantum import Ket, H, Rx, Ry, Rz, CX, sqrt, X, Circuit
from random import uniform
from math import pi


# Ansätze for 1-qubit states
def un_amb_verb_ansatz(p):
    return Ket(p[0])

def amb_verb_ansatz(p):
    return Ket(0) >>  \
        Rx(p[0])

def noun_ansatz(p):
    return Ket(0) >> \
        Rx(p[0]) >> \
        Rz(p[1])


n_qubits_ansatz = 1
n_noun_params = 2
n_amb_params = 1

print(amb_verb_ansatz([uniform(0,1) for i in range(n_amb_params)]).eval())

In [ ]:

# prepare intitial params
binaries = [list(bin(i)[2:]) for i, verb in enumerate(unamb_verb_voc.keys())]
int_binaries = [[0]*(n_qubits_ansatz - len(bi)) + [int(b) for b in bi] for bi in binaries]

n_nouns = len(nouns)
n_amb_verbs = len(ambiguous_verbs)
n_unamb_verbs = len(unamb_verb_voc)

params_unamb_verbs = {verb: int_binaries[i] for i, verb in enumerate(unamb_verb_voc.keys())}
params_nouns = {noun: [uniform(0, 1) for i in range(n_noun_params)] for noun in noun_voc}
params_amb_verbs = {verb: [uniform(0, 1) for i in range(n_amb_params)] for verb in amb_verb_voc}

In [ ]:
params_unamb_verbs

In [ ]:
from discopy import CircuitFunctor, qubit


def F(params_nouns, params_unamb_verbs, params_amb_verbs):
    ar1 = {noun_voc[noun]:noun_ansatz(params_nouns[noun]) for noun in noun_voc}
    ar2 = {unamb_verb_voc[verb]:un_amb_verb_ansatz(params_unamb_verbs[verb]) for verb in unamb_verb_voc}
    ar3 = {amb_verb_voc[verb]:amb_verb_ansatz(params_amb_verbs[verb]) for verb in amb_verb_voc}
    ar = {**ar1, **ar2, **ar3}

    return CircuitFunctor(
        ob = {s: qubit ** 0, n: qubit ** n_qubits_ansatz},
        ar = ar)

print("Circuit for 'register account.':")
circuit = F(params_nouns, params_unamb_verbs, params_amb_verbs)(parsing['register account.'])
circuit.draw(figsize=(6, 6), aspect='auto', draw_type_labels=False)
circuit.eval()

In [ ]:
from pytket.extensions.qiskit import AerBackend, tk_to_qiskit
backend = AerBackend()
result = circuit.eval(backend)

In [ ]:
tk_circ = circuit.to_tk()
print("{}:\n{}\n".format(tk_circ, '\n'.join(map(str, tk_circ))))
print("post selection:\n{}\n".format(tk_circ.post_selection))
print("scalar:\n{}\n".format(tk_circ.scalar))
print("qiskit circuit:")
tk_to_qiskit(tk_circ).draw(output="mpl")

In [ ]:
import numpy as np

# define evaluation function
def evaluate(params_nouns, params_unamb_verbs, params_amb_verbs, sentences, backend=AerBackend(), n_shots=2**10, seed=0):
    circuits = [F(params_nouns, params_unamb_verbs, params_amb_verbs)(parsing[sent]) for sent in sentences]
    results = [Circuit.eval(
                circuit,
                backend=backend,
                n_shots=n_shots,
                seed=seed,
                compilation=backend.default_compilation_pass(2)) for circuit in circuits]
    tensors = [np.abs(result.array)[0] for result in results]
    return tensors

In [ ]:
def reshape_data(params_np):
    ''' Converts numpy array of parameters back to dictionary
    '''
    params_nouns_np = params_np[:n_nouns*n_noun_params].reshape((n_nouns,n_noun_params))
    params_amb_verbs_np = params_np[n_nouns*n_noun_params:].reshape((n_amb_verbs,n_amb_params))

    params_nouns = {word: params_nouns_np[i].tolist() for i, word in enumerate(nouns)}
    params_amb_verbs = {word: params_amb_verbs_np[i].tolist() for i, word in enumerate(amb_verb_voc)}

    return params_nouns, params_amb_verbs

def loss(params_np, sentences=sentences, plausability=plausability):

    # convert np to dict
    params_nouns, params_amb_verbs = reshape_data(params_np)

    return np.mean(np.array([
        (plausability[i] - scalar) ** 2
        for i, scalar in enumerate(evaluate(params_nouns, params_unamb_verbs, params_amb_verbs, sentences))]))

## Fit model with genetic algorithm

In [ ]:
import numpy as np
from geneticalgorithm import geneticalgorithm as ga

algorithm_param = {
    'max_num_iteration': 25,\
    'population_size':5,\
    'mutation_probability':0.1,\
    'elit_ratio': 0.01,\
    'crossover_probability': 0.5,\
    'parents_portion': 0.3,\
    'crossover_type':'uniform',\
    'max_iteration_without_improv':None
}

dimension = n_noun_params*n_nouns + n_amb_params*n_amb_verbs
varbound=np.array([[0,1]]*dimension)

model=ga(function=loss,dimension=dimension, variable_type='real',variable_boundaries=varbound, algorithm_parameters=algorithm_param, function_timeout=100)

model.run()

### Display results after GA

In [ ]:
best_params_nouns, best_params_amb_verbs = reshape_data(model.best_variable)
print(best_params_nouns)
print("\n")
print(best_params_amb_verbs)

In [ ]:
# Wave function of "file"
print(amb_verb_ansatz(best_params_amb_verbs['file']).eval())

In [ ]:
def loss2(params_np, sentences=sentences, plausability=plausability):

    # convert back to dict
    params_nouns, params_amb_verbs = reshape_data(params_np)

    return  {sentences[i]:(plausability[i], round(scalar,4))
        for i, scalar in enumerate(evaluate(params_nouns, params_unamb_verbs, params_amb_verbs, sentences))}

In [ ]:
# Output computed values vs. true values
loss2(model.best_variable)

## Finetune with noisyopt

In [ ]:
import noisyopt
from time import time
i, start = 0, time()

def callback(loss):
    global i
    i += 1
    print("Epoch {} ({:.0f} seconds since start): {}".format(i, time() - start, loss))

result = noisyopt.minimizeSPSA(
    loss, model.best_variable, paired=False, callback=callback, niter=200, a=0.2, c=0.1)

print("Best loss: ", result.fun)

In [ ]:
results = loss2(result.x)

In [ ]:
best_loss = result.fun
print(best_loss)

## Save results


In [ ]:
import datetime, os
folder = './experiments/1_qb_model_' + datetime.datetime.now().strftime("%Y-%m-%d")+ "_best_loss_" + str(round(best_loss,4))
os.makedirs(folder)

In [ ]:
import json

params_nouns, params_amb_verbs = reshape_data(result.x)

fiel_path_params_amb_verbs = os.path.join(folder,'parameters_amb_verbs.json')
fiel_path_params_unamb_verbs = os.path.join(folder,'parameters_unamb_verbs.json')
fiel_path_params_nouns = os.path.join(folder,'params_nouns.json')

fiel_path_results = os.path.join(folder,'results.json')

with open(fiel_path_params_amb_verbs, 'w') as fp:
    json.dump(params_amb_verbs, fp)

with open(fiel_path_params_unamb_verbs, 'w') as fp:
    json.dump(params_unamb_verbs, fp)

with open(fiel_path_params_nouns, 'w') as fp:
    json.dump(params_nouns, fp)

with open(fiel_path_results, 'w') as fp:
    json.dump(results, fp)

In [ ]:
# save the optimizer evolution steps for animation
evolution_genetic = dict()
# deleted _report; see if thsi works
for i, sol in enumerate(model.best_variable_report):
    key = "evo" + str(i+1)
    params_nouns_ev, params_amb_verbs_ev = reshape_data(sol)
    evolution_genetic[key] = {"nouns": params_nouns_ev, "amb_verbs": params_amb_verbs_ev}


file_path_evolution = os.path.join(folder,'params_evolution_genetic.json')
with open(file_path_evolution, 'w') as fp:
    json.dump(evolution_genetic, fp)

In [ ]:
model.best

In [ ]:
# save the optimizer evolution steps for animation
evolution_noisy = dict()
for sol in result.evo:
    params_nouns_ev, params_amb_verbs_ev = reshape_data(result.evo[sol])
    evolution_noisy[sol] = {"nouns": params_nouns_ev, "amb_verbs": params_amb_verbs_ev}

evolution_noisy["loss"] = result.loss_arr

fiel_path_evolution = os.path.join(folder,'params_evolution_noisy.json')
with open(fiel_path_evolution, 'w') as fp:
    json.dump(evolution_noisy, fp)